# ![Company Logo](Thioether-Logo.jpeg)

# Kubernetes and Llamaindex tool

Create a Llama Agent to interact with kubernetes cluster. This is just a preliminary case of tool calling. A sophisticated input model from kubernetes can be used to scan for problems. Why? Because you can and want to put AI in your company profile. This can be done with if elses on the output (structured or unstructured). But there can be cases when plain text output of messages is better handled like a bad/good sentiment by LLM.


## Initialise the LLM

In [15]:
import os
from dotenv import load_dotenv
load_dotenv()

from llama_index.core.agent import ReActAgent
from llama_index.llms.ollama import Ollama
from llama_index.core.tools import FunctionTool
from llama_index.core import Settings

# import logging
# import sys
# logging.basicConfig(stream=sys.stdout, level=logging.WARN)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


# import llama_index.core
# llama_index.core.set_global_handler("simple")


#Settings.llm = Ollama(model="llama3.2:latest",request_timeout=300, temperature=0)

from llama_index.llms.litellm import LiteLLM

liteLLM = LiteLLM(model="groq/llama3-70b-8192",api_key=os.environ['GROQ_API_KEY'])
Settings.llm = liteLLM



In [ ]:
#!pip install kubernetes
#!pip install pandoc

## Define tools to connect to Kubernetes

In [16]:
from kubernetes import client, config
from typing import List
from pydantic import BaseModel,Field

class ClusterEvent(BaseModel):
    """Cluster event messages"""
    message: str = Field(...,description="Cluster Message")

class PodStatus(BaseModel):
    "Represents the pod status"
    namespace:str = Field(..., description="Namespace of the pod")
    podname:str = Field(..., description="Name of the pod")
    status:bool = Field(..., description="Status reflecting if a pod is running.")

# Configs can be set in Configuration class directly or using helper utility
config.load_kube_config()

def get_pod_status() -> List[PodStatus]:
    """Returns the status for all pods in kubernetes cluster."""
    v1 = client.CoreV1Api()
    ret = v1.list_pod_for_all_namespaces(watch=False)
    pods = list[PodStatus]()
    for i in ret.items:
        pods.append(PodStatus(namespace=i.metadata.namespace,podname= i.metadata.name, status = i.status.container_statuses[0].started))
    return pods

def get_cluster_events() -> List[ClusterEvent]:
    """Get events in kubernetes cluster and returns the list of cluster events"""
    v1 = client.CoreV1Api()
    ret = v1.list_event_for_all_namespaces()
    messages = list[ClusterEvent]()
    for i in ret.items:
        messages.append(ClusterEvent(message = i.message))
    return "\n".join(messages)


## Define Function tools and LLM Agent using the tool.

Use the function definitions that can connect to Kubernetes and provide structured information for analysis by LLM.

In [17]:
pod_tool = FunctionTool.from_defaults(fn=get_pod_status)
event_tool = FunctionTool.from_defaults(fn=get_cluster_events)

agent = ReActAgent.from_tools(tools=[pod_tool,event_tool], verbose=True)

## Ask a question

In [18]:
response = agent.chat("Find which pods are not running in a kubernetes cluster?")
display(response.response)

> Running step c158690d-4cd0-4832-8f54-749e7bbab14d. Step input: Find which pods are not running in a kubernetes cluster?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: get_pod_status
Action Input: {'properties': AttributedDict(), 'type': 'object'}
Observation: Error: get_pod_status() got an unexpected keyword argument 'properties'
> Running step e9f0238d-d016-4783-bc23-2952596b8111. Step input: None
Thought: The current language of the user is: English. I need to use a tool to help me answer the question. The tool get_pod_status does not require any input.
Action: get_pod_status
Action Input: {}
Observation: [PodStatus(namespace='kube-system', podname='coredns-7b98449c4-2xcxg', status=True), PodStatus(namespace='kube-system', podname='helm-install-traefik-6jv8m', status=False), PodStatus(namespace='kube-system', podname='helm-install-traefik-crd-nzgh2', status=False), PodStatus(namespace='kube-system', podname='local

'The pods that are not running are: helm-install-traefik-6jv8m, helm-install-traefik-crd-nzgh2.'

## Ask a question

In [ ]:
#agent.chat("Analyse the events in kubernetes cluster and suggest if any error requires attention.")